# htrc-vectorize

This notebook demonstrates how to use HTRC extracted features to build a word2vec model using Doc2Vec. 

Jed Dobson<br>
James.E.Dobson@Dartmouth.EDU<br>
http://www.dartmouth.edu/~jed<br>
October 2020<br>
 
This file is part of the htrc-vector-project, begun in June 2020 with Catherine Parnell. 

<b>Repository</b>:<br>
https://github.com/jeddobson/htrc-vector-project

### Why use HTRC Features?

The <a href="https://analytics.hathitrust.org/features">HTRC Extracted Features dataset</a> include over 17 million volumes. This includes works presently still protected by copyright. These features can be distributed because they are for non-consumptive use. They are designed to be computer rather than human readable. This means that you can model large number of texts including texts from the twenty and twenty-first century. Document and page-based features are available. The <a href="https://github.com/htrc/htrc-feature-reader">HTRC feature-reader package</a> for Python enables easy access to these features. These features include tokens and their repititions on a page-by-page basis. Word order is lost (thus the page is no longer human readable). The format used limits their use, however, and many popular applications of text mining and machine learning expect preserved word order. The popular Skipgram model used by <a href="https://github.com/tmikolov/word2vec">word2vec</a>, for example, learns by predicting words within a window surrounding a target word. 
    
### Doc2Vec
This notebook uses <a href="https://radimrehurek.com/gensim/models/doc2vec.html">Doc2Vec</a> to train a model for words appearing within a much larger window. Doc2Vec has been used to produce vectors from paragraph-length text sources. These vectors are then used for classification and other applications.

### Tunable and Limitations
This example notebook uses a set of Toni Morrison novels found in the HATHI Trust archive. This is a much smaller dataset than we've used in other experiments. Vector models like word2vec generally require larger numbers of text sources (we've trained on 30GB archives). 

<b>Text Sources</b>
- input size: Depending on what you want to model, you will most likely want to work with more than words written by a single author. We successfully used this approach with thousands of texts across multiple genres.
- preprocessing: This notebook does not provide any preprocessing and simply imports all extracted features. You may want to remove some words with high frequent use (i.e., "stopwords"). Results may vary.

<b>Execution</b>
In real execution, you'll want to separate the several phases of this workflow. You can predownload the features and load these locally (this script will download as needed). To manage run time, you may also want to process each text individually. We produced a CSV file with one line for each page of the text. These were then concatenated (and compressed with bzip) and converted into a TaggedDocument before running doc2vec.

<b>doc2vec</b>
- window: This is the primary tunable. The smaller this variable the more likely you are to find similar word vectors based on alphabetical ordering (the tokens are processed in alphabetical ordering, as received from the feature-reader). The window size should approximate the typical page length.
- min_count: Most errors introduced during digitization or creation of the extracted features should be removed with a small integer used here. 


In [2]:
from htrc_features import FeatureReader, utils  
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.models.keyedvectors as kv

In [3]:
# download data
documents = [
    ["The Bluest Eye","uc1.32106018657251"],
    ["Song of Solomon","mdp.39015032749130"],
    ["Sula","uc1.32106019072633"],
    ["Tar Baby","uc1.32106005767956"],
    ["Jazz","ien.35556029664190"],
    ["Beloved","mdp.49015003142743"],
    ["Paradise","mdp.39015066087613"],
    ["A Mercy","mdp.39076002787351"]
]

In [4]:
#for d in documents:
#    path = utils.id_to_rsync(d[1])
#    path = "data.analytics.hathitrust.org::features-2020.03/" + path
#    !rsync -av {path} .

In [5]:
# This function extracts individual pages and create string of words from tokens
# Word order is lost from HTRC features. This creates page length strings by
# multiplying tokens for each appearance. Thus, token the with count 2 will 
# appear as "the the" in the returned string.

def get_pages(document):
    fr = FeatureReader([document])
    vol = next(fr.volumes())
    ptc = vol.tokenlist(pos=False, case=False).reset_index().drop(['section'], axis=1)
    page_list = set(ptc['page'])
    
    rows=list()
    for page in page_list:
        page_data = str()
        
        # operate on each token
        for page_tokens in ptc.loc[ptc['page'] == page].iterrows():
            if page_tokens[1][1].isalpha():
                page_data += (' '.join([page_tokens[1][1]] * page_tokens[1][2])) + " "

        # Doc2Vec needs comma separated list of words
        rows.append(page_data.split())
    return rows

In [6]:
# Process downloaded features and store as TaggedDocument with a tag for page number
# This tage is required for Doc2Vec and would normally be based on paragraphs but we
# can only operate on pages of data from HTRC extracted features
#

pages = list()
for d in documents:
    for page in get_pages(d[1]):
        pages.append(page)

# convert to TaggedDocument
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(pages)]

In [7]:
print("creating model")
model = Doc2Vec(tagged_data, 
                dm=1, # operate on "paragraphs" (pages) with distributed memory model
                vector_size=300, # larger vector size might produce better results
                min_count=5, # drop words with very few repetitions
                window=150, # larger window size needed because of extracted features
                workers=2)

print("saving word2vec model")
model.save_word2vec_format("doc2vec-morrison-novels.w2v")

creating model
saving word2vec model


In [8]:
# load and verify
model =  kv.KeyedVectors.load_word2vec_format("doc2vec-morrison-novels.w2v")

In [9]:
model.most_similar(["memory"],topn=25)

[('possibility', 0.9636703133583069),
 ('simply', 0.938510000705719),
 ('quite', 0.9355024099349976),
 ('possible', 0.9313812255859375),
 ('ruby', 0.9290497899055481),
 ('parents', 0.9280668497085571),
 ('losing', 0.9276642799377441),
 ('reduced', 0.9182644486427307),
 ('sold', 0.9174015522003174),
 ('slaughter', 0.9172781705856323),
 ('rare', 0.9135681986808777),
 ('rear', 0.913515031337738),
 ('shriveled', 0.9132422804832458),
 ('rain', 0.9129200577735901),
 ('malice', 0.9115512371063232),
 ('pond', 0.9083991646766663),
 ('refused', 0.9077808856964111),
 ('passed', 0.9076093435287476),
 ('rejection', 0.9070072770118713),
 ('transferred', 0.8983979225158691),
 ('soda', 0.897973895072937),
 ('presence', 0.8973550200462341),
 ('sunrise', 0.896835207939148),
 ('sign', 0.8936558961868286),
 ('met', 0.8929139375686646)]